## Create a training YAML file

In [ ]:
import os
import json
import shutil

orig_ann_train = "/content/annotation/train"
orig_ann_val = "/content/annotation/valid"
orig_img_train = "/content/train"
orig_img_val = "/content/valid"

yolo_base = "/content/bdd100k_yolo"
yolo_img_train = os.path.join(yolo_base, "images/train")
yolo_img_val = os.path.join(yolo_base, "images/val")
yolo_label_train = os.path.join(yolo_base, "labels/train")
yolo_label_val = os.path.join(yolo_base, "labels/val")

classes = ["traffic sign"]

for d in [yolo_img_train, yolo_img_val, yolo_label_train, yolo_label_val]:
    os.makedirs(d, exist_ok=True)

def convert_to_yolo(json_path, img_dir, yolo_label_dir):
    with open(json_path, "r") as f:
        data = json.load(f)

    img_width, img_height = 1280, 720  

    txt_filename = os.path.splitext(os.path.basename(json_path))[0] + ".txt"
    txt_path = os.path.join(yolo_label_dir, txt_filename)

    with open(txt_path, "w") as f_out:
        for obj in data:
            category = obj["category"]
            if category not in classes:
                continue

            class_id = classes.index(category)
            box = obj["box2d"]
            x1, y1, x2, y2 = box["x1"], box["y1"], box["x2"], box["y2"]

            # YOLO format
            x_center = ((x1 + x2) / 2) / img_width
            y_center = ((y1 + y2) / 2) / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height
            f_out.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Train data transform
for json_file in os.listdir(orig_ann_train):
    json_path = os.path.join(orig_ann_train, json_file)
    convert_to_yolo(json_path, orig_img_train, yolo_label_train)

    img_file = json_file.replace(".json", ".jpg")
    src_img_path = os.path.join(orig_img_train, img_file)
    dst_img_path = os.path.join(yolo_img_train, img_file)
    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)

# Validation data transform
for json_file in os.listdir(orig_ann_val):
    json_path = os.path.join(orig_ann_val, json_file)
    convert_to_yolo(json_path, orig_img_val, yolo_label_val)

    img_file = json_file.replace(".json", ".jpg")
    src_img_path = os.path.join(orig_img_val, img_file)
    dst_img_path = os.path.join(yolo_img_val, img_file)
    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)

# generate dataset.yaml 
yaml_content = f"""
path: {yolo_base}
train: images/train
val: images/val
nc: {len(classes)}
names: {classes}
"""

yaml_path = os.path.join(yolo_base, "dataset.yaml")
with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())


### train

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')
results = model.train(data='/content/bdd100k_yolo/dataset.yaml', epochs=50)
results = model.val()